![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [105]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv('bank_marketing.csv')
print(df.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')


In [106]:
client_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing','loan']]
client_df = client_df.rename(columns={'client_id': 'id'})
client_df['education'] = client_df['education'].str.replace('.', '_')
client_df['education'].fillna('unknown', inplace=True)
print(client_df.head())
client_df.to_csv('client.csv', index = False)

   id  age        job  marital    education credit_default housing loan
0   0   56  housemaid  married     basic_4y             no      no   no
1   1   57   services  married  high_school        unknown      no   no
2   2   37   services  married  high_school             no     yes   no
3   3   40     admin.  married     basic_6y             no      no   no
4   4   56   services  married  high_school             no      no  yes


In [107]:
campaign_df = df[['client_id','campaign','duration','pdays','previous','poutcome','y',]]
campaign_df = campaign_df.rename(columns={'duration': 'contact_duration','y':'campaign_outcome',
'poutcome':'previous_outcome',                                          'previous':'previous_campaign_contacts',
'campaign':'number_contacts'})
campaign_df.insert(0, 'campaign_id', 1)
campaign_df = campaign_df.replace({'previous_outcome': 'nonexistent','campaign_outcome': 'nonexistent'}, np.nan)
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace({'success':1,'failure':0})
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].replace({'yes':1,'no':0})
campaign_df['last_contact_date']= '2022-'+df['month'] +'-'+ df['day'].astype(str)
campaign_df.head()
campaign_df.to_csv('campaign.csv',index=False)



In [108]:
economics_df = df[['client_id','emp_var_rate','cons_price_idx','euribor3m',
                  'nr_employed']]
economics_df = economics_df.rename(columns={'euribor3m':'euribor_three_months','nr_employed':'number_employed'})
economics_df.head()
economics_df.to_csv('economics.csv',index = False)



In [109]:
print(client_df.columns)
print(campaign_df.columns)
print(economics_df.columns)

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')
Index(['campaign_id', 'client_id', 'number_contacts', 'contact_duration',
       'pdays', 'previous_campaign_contacts', 'previous_outcome',
       'campaign_outcome', 'last_contact_date'],
      dtype='object')
Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


In [110]:
client_table = '''
CREATE TABLE client(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN,

);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
'''

campaign_table = '''
    CREATE TABLE campaign(
        campaign_id SERIAL PRIMARY KEY,
        client_id SERIAL references client (id),
        number_contacts INTEGER,
        contact_duration INTEGER,
        pdays INTEGER,
        previous_campaign_contacts INTEGER,
        previous_outcome BOOLEAN,
        campaign_outcome BOOLEAN,
        last_contact_date DATE,
    );
     \copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
'''

economics_table= '''
    CREATE TABLE economics(
        client_id SERIAL references client (id),
        emp_var_rate FLOAT,
        cons_price_idx FLOAT,
        euribor_three_months FLOAT,
        number_employed FLOAT,
    );
      \copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
'''